## 신청 방법 안내 BLOG
https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221341051478&proxyReferer=https:%2F%2Fwww.google.com%2F

## 신청 사이트
https://www.data.go.kr/dataset/15012690/openapi.do

## 인증키
YBicSRtx%2Fllk3emEF2RPAk6gJJhbRaeaNr2MoASOHSBXZuZkrR%2Bi9vAqV6aMnK1m%2FBqxH6%2BUKD0Ac%2BN%2F9oP47A%3D%3D

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import datetime
import os

In [2]:
def get_request_query(url, operation, params, serviceKey):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

## 날짜 유혀성 확인
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

In [3]:
## 초기설정
start_year = 2015
end_year = 2020
start_month = 1
end_month = 12

In [4]:
# 요청 URL과 오퍼레이션
URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
OPERATION = 'getHoliDeInfo' # 국경일 + 공휴일 정보 조회 오퍼레이션

# 파라미터
SERVICEKEY = 'YBicSRtx%2Fllk3emEF2RPAk6gJJhbRaeaNr2MoASOHSBXZuZkrR%2Bi9vAqV6aMnK1m%2FBqxH6%2BUKD0Ac%2BN%2F9oP47A%3D%3D'
solYear  = '2018'  # 연도
solMonth = '09'   # 월
PARAMS = {'solYear':solYear, 'solMonth':solMonth}

In [5]:
merge_dict = {
    'date':[],
    'holiday':[],
}

for temp_year in tqdm(range(start_year, end_year+1), desc='year'):
    temp_year = str(temp_year)
    for temp_month in tqdm(range(start_month, end_month+1), desc='month'):
        temp_month = str(temp_month).zfill(2)
        
        PARAMS = {'solYear':temp_year, 'solMonth':temp_month}
        while(True):
            request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
            response = requests.get(url=request_query)
            if True == response.ok:
                break
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        for item in soup.find_all('item'):
            temp_date = item.find('locdate').text
            temp_date = "{}-{}-{}".format(temp_date[:4], temp_date[4:6], temp_date[6:])
            
            ## 날짜 형식 확인
            if validate(temp_date):
                merge_dict['date'].append(temp_date)
                merge_dict['holiday'].append(1)

In [6]:
merge_df = pd.DataFrame(merge_dict)

In [7]:
## 타입변환
merge_df['date'] = pd.to_datetime(merge_df['date'], format='%Y-%m-%d')
merge_df = merge_df.drop_duplicates()

In [8]:
merge_df = merge_df.set_index('date')

In [9]:
## 날짜 계산
all_days = pd.date_range(merge_df.index.min(), merge_df.index.max(), freq='D')
merge_df = merge_df.reindex(all_days)

In [10]:
merge_df = merge_df.fillna(0)

In [11]:
## index를 원래 자리로
merge_df = merge_df.reset_index().rename(columns={"index": "date"})

In [13]:
merge_df.to_csv('holiday_total.csv', index=None)